# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Develop-Thermodynamic-kinetic-Maximum-Entropy-Model" data-toc-modified-id="Develop-Thermodynamic-kinetic-Maximum-Entropy-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Develop Thermodynamic-kinetic Maximum Entropy Model</a></div><div class="lev2 toc-item"><a href="#Reactions-File" data-toc-modified-id="Reactions-File-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Reactions File</a></div><div class="lev3 toc-item"><a href="#Read-the-file-into-a-dataframe-and-create-a-stoichiometric-matrix" data-toc-modified-id="Read-the-file-into-a-dataframe-and-create-a-stoichiometric-matrix-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Read the file into a dataframe and create a stoichiometric matrix</a></div><div class="lev2 toc-item"><a href="#Calculate-Standard-Free-Energies-of-Reaction" data-toc-modified-id="Calculate-Standard-Free-Energies-of-Reaction-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calculate Standard Free Energies of Reaction</a></div><div class="lev3 toc-item"><a href="#Output-the-Standard-Reaction-Free-Energies-for-use-in-a-Boltzmann-Simulation" data-toc-modified-id="Output-the-Standard-Reaction-Free-Energies-for-use-in-a-Boltzmann-Simulation-122"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Output Standard Reaction Free Energies for Later Use</a></div><div class="lev2 toc-item"><a href="#Set-Fixed-Concentrations/Boundary-Conditions" data-toc-modified-id="Set-Fixed-Concentrations/Boundary-Conditions-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Set Fixed Concentrations/Boundary Conditions</a></div><div class="lev2 toc-item"><a href="#Prepare-model-for-optimization" data-toc-modified-id="Prepare-model-for-optimization-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Prepare model for optimization</a></div><div class="lev1 toc-item"><a href="#Nonlinear-Least-Squares-Optimization-of-Concentrations" data-toc-modified-id="Nonlinear-Least-Squares-Optimization-of-Concentrations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Nonlinear Least Squares Optimization of Concentrations</a></div><div class="lev2 toc-item"><a href="#Apply-Regulation-and-Optimize" data-toc-modified-id="Apply-Regulation-and-Optimize-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Apply MCA Regulation Method and Optimize</a></div><div class="lev1 toc-item"><a href="#ODE-Simulations" data-toc-modified-id="ODE-Simulations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ODE Simulations</a></div><div class="lev2 toc-item"><a href="#Calculate-Rate-Constants" data-toc-modified-id="Calculate-Rate-Constants-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Calculate Rate Constants</a></div><div class="lev2 toc-item"><a href="#ODE-Solvers:-Python-interface-using-libroadrunner-to-Sundials/CVODE" data-toc-modified-id="ODE-Solvers:-Python-interface-using-libroadrunner-to-Sundials/CVODE-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Reinforcement Learning Based Regulation</a></div>

# Develop Thermodynamic-kinetic Maximum Entropy Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import sys
import re
import os

from scipy.optimize import least_squares
from IPython.core.display import display
from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

%matplotlib inline
T = 298.15
R = 8.314e-03
RT = R*T
N_avogadro = 6.022140857e+23
VolCell = 1.0e-15
Concentration2Count = N_avogadro * VolCell
concentration_increment = 1/(N_avogadro*VolCell)

use_experimental_data=True

In [2]:
cwd = os.getcwd()
os.chdir('../')
cwd_up = os.getcwd()
os.chdir(cwd)

sys.path.insert(0, cwd_up+'/Basic_Functions')
import max_entropy_functions as me

display(cwd)

'C:\\Users\\samuel_britton\\Documents\\cannon\\Final_Pathways\\Python_Notebook_Regulation\\gluconeogenesis'

## Reactions File

In [3]:
with open( cwd + '/Gluconeogenesis.dat', 'r') as f:
    print(f.read())    

//
REACTION	 GLUCOSE_6_PHOSPHATASE
LEFT BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL
RIGHT BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DGZERO -17.057763138721384
DGZERO StdDev 0.7152372520174535
DGZERO-UNITS    KJ/MOL
//
REACTION	 PGI
LEFT D-FRUCTOSE_6-PHOSPHATE:CYTOSOL
RIGHT BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DGZERO 2.524005856552094
DGZERO StdDev 0.5967754160874962
DGZERO-UNITS    KJ/MOL
//
REACTION	 FBP
LEFT H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL
RIGHT D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DGZERO 0.0
DGZERO StdDev 0.0
DGZERO-UNITS    KJ/MOL
//
REACTION	 FBA
LEFT	GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL
RIGHT	D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DG

### Read the file into a dataframe and create a stoichiometric matrix

In [8]:
fdat = open('Gluconeogenesis.dat', 'r')

left ='LEFT'
right = 'RIGHT'
left_compartment = 'LEFT_COMPARTMENT'
right_compartment = 'RIGHT_COMPARTMENT'
enzyme_level = 'ENZYME_LEVEL'
deltag0 = 'DGZERO'
deltag0_sigma = 'DGZERO StdDev'
same_compartment = 'Same Compartment?'
full_rxn = 'Full Rxn'

reactions = pd.DataFrame(index=[],columns=[left, right, left_compartment, right_compartment, enzyme_level, deltag0, deltag0_sigma, same_compartment,full_rxn])
reactions.index.name='REACTION'
S_matrix = pd.DataFrame(index=[],columns=[enzyme_level])
S_matrix.index.name='REACTION'

for line in fdat:
    if (line.startswith('REACTION')):
        rxn_name = line[9:-1].lstrip()
        S_matrix.loc[rxn_name,enzyme_level] = 1.0
        reactions.loc[rxn_name,enzyme_level] = 1.0
    if (re.match("^LEFT\s",line)):
        line = line.upper()
        left_rxn = line[4:-1].lstrip()
        left_rxn = re.sub(r'\s+$', '', left_rxn) #Remove trailing white space
        reactions.loc[rxn_name,left] = left_rxn
    elif (re.match('^RIGHT\s',line)):
        line = line.upper()
        right_rxn = line[5:-1].lstrip()
        right_rxn = re.sub(r'\s+$', '', right_rxn) #Remove trailing white space
        reactions.loc[rxn_name,right] = right_rxn    
    elif (line.startswith(left_compartment)):
        cpt_name = line[16:-1].lstrip()
        reactions.loc[rxn_name,left_compartment] = cpt_name
        reactants = re.split(' \+ ',left_rxn)
        for idx in reactants:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(-1.0);
                molecule = values[0]; 
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;
    elif (line.startswith(right_compartment)):
        cpt_name = line[17:-1].lstrip()
        reactions.loc[rxn_name,right_compartment] = cpt_name
        products = re.split(' \+ ',right_rxn)
        for idx in products:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(1.0);
                molecule = values[0];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;
    elif (re.match("^ENZYME_LEVEL\s", line)):
        level = line[12:-1].lstrip()
        reactions.loc[rxn_name,enzyme_level] = float(level)
        S_matrix.loc[rxn_name,enzyme_level] = float(level)       
    elif re.match('^COMMENT',line):
        continue
    elif re.match(r'//',line):
        continue
    elif re.match('^#',line):
        continue
        
fdat.close()
S_matrix.fillna(0,inplace=True)
S_active = S_matrix[S_matrix[enzyme_level] > 0.0]
active_reactions = reactions[reactions[enzyme_level] > 0.0]
del S_active[enzyme_level]

# Delete any columns/metabolites that have all zeros in the S matrix:
S_active = S_active.loc[:, (S_active != 0).any(axis=0)]
np.shape(S_active.values)
display(S_active.shape)
display(S_active)
reactions[full_rxn] = reactions[left] + ' = ' + reactions[right]

(11, 19)

,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL,ORTHOPHOSPHATE:CYTOSOL,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",GLYCERONE_PHOSPHATE:CYTOSOL,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,NADH:CYTOSOL,NAD+:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,ATP:CYTOSOL,ADP:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,PHOSPHOENOLPYRUVATE:CYTOSOL,OXALOACETATE:CYTOSOL,CO2:CYTOSOL,PYRUVATE:CYTOSOL
REACTION,,,,,,,,,,,,,,,,,,,
GLUCOSE_6_PHOSPHATASE,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PGI,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBP,0.0,-1.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBA,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TPI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GAPD,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PGK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0
PGM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
ENO,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0


In [9]:
for idx in reactions.index:
    boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
    if re.search(':',boltzmann_rxn_str):
        all_cmprts = re.findall(':\S+', boltzmann_rxn_str)
        [s.replace(':', '') for s in all_cmprts] # remove all the ':'s 
        different_compartments = 0
        for cmpt in all_cmprts:
            if not re.match(all_cmprts[0],cmpt):
                different_compartments = 1
        if ((not different_compartments) and (reactions[left_compartment].isnull or reactions[right_compartment].isnull)):
            reactions.loc[idx,left_compartment] = cmpt
            reactions.loc[idx,right_compartment] = cmpt
            reactions.loc[idx,same_compartment] = True
        if different_compartments:
            reactions.loc[idx,same_compartment] = False
    else:
        if (reactions.loc[idx,left_compartment] == reactions.loc[idx,right_compartment]):
            reactions.loc[idx,same_compartment] = True
        else:
            reactions.loc[idx,same_compartment] = False
display(reactions)                        

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
REACTION,,,,,,,,,
GLUCOSE_6_PHOSPHATASE,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOS...
PGI,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL = BETA-D-GLUCOS...
FBP,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHAT...,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTO..."
FBA,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...
TPI,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,GLYCERONE_PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL = GLYCERO...
GAPD,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...
PGK,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + ADP:...,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL = ...
PGM,2-PHOSPHO-D-GLYCERATE:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,2-PHOSPHO-D-GLYCERATE:CYTOSOL = 3-PHOSPHO-D-GL...
ENO,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL = 2-...


## Calculate Standard Free Energies of Reaction 

In [10]:
import sys
sys.path.insert(0, cwd_up+'/equilibrator-api-v0.1.8/build/lib')

from equilibrator_api import *
from equilibrator_api.reaction_matcher import ReactionMatcher
reaction_matcher = ReactionMatcher()
eq_api = ComponentContribution(pH=7.0, ionic_strength=0.25)  # loads data
for idx in reactions.index:
    print(idx, flush=True)
    boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
    full_rxn_str_no_cmprt = re.sub(':\S+','', boltzmann_rxn_str)
    print(full_rxn_str_no_cmprt)
    full_rxn_str_no_cmprt = re.sub('BETA-D-GLUCOSE','D-GLUCOSE',full_rxn_str_no_cmprt )
    rxn = reaction_matcher.match(full_rxn_str_no_cmprt)
    if not rxn.check_full_reaction_balancing():
        print('Reaction %s is not balanced:\n %s\n' % (idx, full_rxn_str_no_cmprt), flush=True)
    dG0_prime, dG0_uncertainty = eq_api.dG0_prime(rxn)
    display(dG0_prime, dG0_uncertainty)
    reactions.loc[idx,deltag0] = dG0_prime
    reactions.loc[idx,deltag0_sigma] = dG0_uncertainty

    reactions.loc['PYRt2m',deltag0] = -RT*np.log(10)
display(reactions)

GLUCOSE_6_PHOSPHATASE
BETA-D-GLUCOSE-6-PHOSPHATE + H2O = BETA-D-GLUCOSE + ORTHOPHOSPHATE


-8.998188834383427

0.6370893718226682

PGI
D-FRUCTOSE_6-PHOSPHATE = BETA-D-GLUCOSE-6-PHOSPHATE


-2.5220568042852847

...

7.639128086053029

Pyruvate_Carboxylase
ATP + PYRUVATE + CO2 + H2O = ADP + ORTHOPHOSPHATE + OXALOACETATE


-0.7958250266501636

7.604190202200161

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
REACTION,,,,,,,,,
GLUCOSE_6_PHOSPHATASE,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,-8.99819,0.637089,True,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOS...
PGI,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,-2.52206,0.596775,True,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL = BETA-D-GLUCOS...
FBP,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHAT...,:CYTOSOL,:CYTOSOL,1,-9.67092,0.818386,True,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTO..."
FBA,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",:CYTOSOL,:CYTOSOL,1,-21.4506,0.87227,True,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...
TPI,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,GLYCERONE_PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,-5.49798,0.753116,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL = GLYCERO...
GAPD,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,:CYTOSOL,:CYTOSOL,1,-5.24202,0.895659,True,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...
PGK,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + ADP:...,:CYTOSOL,:CYTOSOL,1,18.5083,0.889982,True,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL = ...
PGM,2-PHOSPHO-D-GLYCERATE:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,-4.17874,0.65542,True,2-PHOSPHO-D-GLYCERATE:CYTOSOL = 3-PHOSPHO-D-GL...
ENO,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1,4.0817,0.734193,True,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL = 2-...


### Output Standard Reaction Free Energies for Later Use

In [11]:
reaction_file = open('Gluconeogenesis.keq', 'w')
for y in reactions.index:
    print('%s\t%e' % (y, np.exp(-reactions.loc[y,'DGZERO']/RT)),file=reaction_file)
reaction_file.close()    

reaction_file.close()    
display((S_active.columns))

Index(['BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL', 'H2O:CYTOSOL',
       'BETA-D-GLUCOSE:CYTOSOL', 'ORTHOPHOSPHATE:CYTOSOL',
       'D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', 'D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL',
       'GLYCERONE_PHOSPHATE:CYTOSOL', 'D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL',
       '3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL', 'NADH:CYTOSOL',
       'NAD+:CYTOSOL', '3-PHOSPHO-D-GLYCERATE:CYTOSOL', 'ATP:CYTOSOL',
       'ADP:CYTOSOL', '2-PHOSPHO-D-GLYCERATE:CYTOSOL',
       'PHOSPHOENOLPYRUVATE:CYTOSOL', 'OXALOACETATE:CYTOSOL', 'CO2:CYTOSOL',
       'PYRUVATE:CYTOSOL'],
      dtype='object')

## Set Fixed Concentrations/Boundary Conditions

In [12]:
### Set Fixed Concentrations/Boundary Conditions
conc = 'Conc'
variable = 'Variable'
conc_exp = 'Conc_Experimental'
metabolites = pd.DataFrame(index = S_active.columns, columns=[conc,conc_exp,variable])
metabolites[conc] = 0.001
metabolites[variable] = True

metabolites.loc['2-PHOSPHO-D-GLYCERATE:CYTOSOL', conc] = 1.98e-01
metabolites.loc['PHOSPHOENOLPYRUVATE:CYTOSOL', conc] = 8.50e-02
metabolites.loc['OXALOACETATE:CYTOSOL', conc] = 1.000000e-03
metabolites.loc['3-PHOSPHO-D-GLYCERATE:CYTOSOL', conc] = 1.30e+01
metabolites.loc['3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL', conc] = 1.56e+00
metabolites.loc['GLYCERONE_PHOSPHATE:CYTOSOL', conc] = 4.18E-05
metabolites.loc['D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL', conc] = 7.41E-07
metabolites.loc['D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', conc] = 3.18E-01
metabolites.loc['D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL', conc] = 9.60E-02
metabolites.loc['BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL', conc] = 3.80E-03

# Set the fixed metabolites:
metabolites.loc['PYRUVATE:CYTOSOL', conc] = 1.00E-03
metabolites.loc['PYRUVATE:CYTOSOL', variable] = False
metabolites.loc['BETA-D-GLUCOSE:CYTOSOL', conc] = 1.00E-11
metabolites.loc['BETA-D-GLUCOSE:CYTOSOL',variable] = False
metabolites.loc['ATP:CYTOSOL', conc] = 9.60E-03
metabolites.loc['ATP:CYTOSOL',variable] = False
metabolites.loc['ADP:CYTOSOL', conc] = 5.60E-04
metabolites.loc['ADP:CYTOSOL',variable] = False

# Low Ratio from NADP/NADPH
# NADP/NADPH = 2.1×10-6/ 1.2 ×10-4
# nadh/nad 2.10E-06 / 1.20E-04
metabolites.loc['NADH:CYTOSOL', conc] = 1.20E-04
metabolites.loc['NADH:CYTOSOL',variable] = False
metabolites.loc['NAD+:CYTOSOL', conc] = 2.10E-06
metabolites.loc['NAD+:CYTOSOL',variable] = False


metabolites.loc['ORTHOPHOSPHATE:CYTOSOL', conc] = 2.00E-06
metabolites.loc['ORTHOPHOSPHATE:CYTOSOL',variable] = False
metabolites.loc['CO2:CYTOSOL', conc] = 1.00E-04
metabolites.loc['CO2:CYTOSOL',variable] = False
metabolites.loc['H2O:CYTOSOL', conc] = 55
metabolites.loc['H2O:CYTOSOL',variable] = False    


#When loading experimental concentrations, first copy current 
#rule of thumb then overwrite with data values. Here we simply use rule of thumb concentrations.
metabolites[conc_exp] = metabolites[conc]

## Prepare model for optimization

- Adjust S Matrix to use only reactions with activity > 0, if necessary.
- Water stoichiometry in the stiochiometric matrix needs to be set to zero since water is held constant.
- The initial concentrations of the variable metabolites are random.
- All concentrations are changed to log counts.
- Equilibrium constants are calculated from standard free energies of reaction.
- R (reactant) and P (product) matrices are derived from S.

In [13]:
# Make sure all the indices and columns are in the correct order:

nvariables = metabolites[metabolites[variable]].count()
nvar = nvariables[variable]

metabolites.sort_values(by=variable, axis=0,ascending=False, inplace=True,)


# ## Prepare model for optimization

# - Adjust S Matrix to use only reactions with activity > 0, if necessary.
# - Water stoichiometry in the stiochiometric matrix needs to be set to zero since water is held constant.
# - The initial concentrations of the variable metabolites are random.
# - All concentrations are changed to log counts.
# - Equilibrium constants are calculated from standard free energies of reaction.
# - R (reactant) and P (product) matrices are derived from S.

# Make sure all the indices and columns are in the correct order:
active_reactions = reactions[reactions[enzyme_level] > 0.0]

Sactive_index = S_active.index

#active_reactions = reactions[reactions[enzyme_level] > 0.0]
#Sactive_index = S_active.index

active_reactions.reindex(index = Sactive_index, copy = False)
S_active = S_active.reindex(columns = metabolites.index, copy = False)
S_active['H2O:CYTOSOL'] = 0

where_are_NaNs = np.isnan(S_active)
S_active[where_are_NaNs] = 0
S_mat = S_active.values

Keq_constant = np.exp(-active_reactions[deltag0].astype('float')/RT)
Keq_constant = Keq_constant.values

P_mat = np.where(S_mat>0,S_mat,0)
R_mat = np.where(S_mat<0, S_mat, 0)

mu0 = 1 #Dummy parameter for now; reserved for free energies of formation

conc_type=conc
if (use_experimental_data):
    print("USING EXPERIMENTAL DATA")
    conc_type=conc_exp
    
    
variable_concs = np.array(metabolites[conc_type].iloc[0:nvar].values, dtype=np.float64)
v_log_concs = -10 + 10*np.random.rand(nvar) #Vary between 1 M to 1.0e-10 M
v_concs = np.exp(v_log_concs)
v_log_counts_stationary = np.log(v_concs*Concentration2Count)
v_log_counts = v_log_counts_stationary

fixed_concs = np.array(metabolites[conc_type].iloc[nvar:].values, dtype=np.float64)
fixed_counts = fixed_concs*Concentration2Count
f_log_counts = np.log(fixed_counts)

complete_target_log_counts = np.log(Concentration2Count * metabolites[conc_type].values)
target_v_log_counts = complete_target_log_counts[0:nvar]
target_f_log_counts = complete_target_log_counts[nvar:]

delta_increment_for_small_concs = np.zeros(metabolites[conc_type].values.size);

variable_concs_begin = np.array(metabolites[conc_type].iloc[0:nvar].values, dtype=np.float64)
display(metabolites)


USING EXPERIMENTAL DATA


,Conc,Conc_Experimental,Variable
BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,3.800000e-03,3.800000e-03,True
OXALOACETATE:CYTOSOL,1.000000e-03,1.000000e-03,True
D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,3.180000e-01,3.180000e-01,True
"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",9.600000e-02,9.600000e-02,True
GLYCERONE_PHOSPHATE:CYTOSOL,4.180000e-05,4.180000e-05,True
D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,7.410000e-07,7.410000e-07,True
3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,1.560000e+00,1.560000e+00,True
PHOSPHOENOLPYRUVATE:CYTOSOL,8.500000e-02,8.500000e-02,True
2-PHOSPHO-D-GLYCERATE:CYTOSOL,1.980000e-01,1.980000e-01,True
3-PHOSPHO-D-GLYCERATE:CYTOSOL,1.300000e+01,1.300000e+01,True


In [15]:
#initial test
E_regulation = np.ones(Keq_constant.size) # This is the vector of enzyme activities, Range: 0 to 1.

#Test optimization routine without regulation
res_lsq1 = least_squares(me.derivatives, v_log_counts, method='lm',xtol=1e-15, args=(f_log_counts, mu0, S_mat, R_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation))
rxn_flux = me.oddsDiff(res_lsq1.x, f_log_counts, mu0, S_mat, R_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation)

print("Steady State Metabolite Concentrations")
display(res_lsq1.x)
print("Flux")
display(rxn_flux)



Steady State Metabolite Concentrations


array([-0.76237043, 18.9332732 , -0.893601  ,  3.18494853, -1.6253171 ,
       -2.95709017, -0.57386028,  8.36125623,  5.2648756 ,  5.50090307])

Flux


array([2.0137094 , 2.0137094 , 2.0137094 , 2.0137094 , 2.0137094 ,
       4.02741881, 4.02741881, 4.02741881, 4.02741881, 4.02741881,
       4.02741881])

## Apply MCA Regulation Method and Optimize

In [16]:
#initialize variables
delta_S = np.ones(Keq_constant.size)
delta_S_metab = np.ones(metabolites.size)
E_regulation=np.ones(Keq_constant.size)

v_log_counts = np.log(v_concs*Concentration2Count)

ipolicy=1 #USE 'local'(1) or 'unrestricted'(2)
#ipolicy=1

React_Choice=0

v_log_counts = np.log(variable_concs_begin * Concentration2Count)
i=0
while( (i < 500) ):

    res_lsq = least_squares(me.derivatives, v_log_counts, method='lm',xtol=1e-15, args=(f_log_counts, mu0, S_mat, R_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation))
    v_log_counts = res_lsq.x
    log_metabolites = np.append(v_log_counts, f_log_counts)
        
    #make calculations to regulate
    rxn_flux = me.oddsDiff(v_log_counts, f_log_counts, mu0, S_mat, R_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation)
        
    KQ_f = me.odds(log_metabolites, mu0,S_mat, R_mat, P_mat, delta_increment_for_small_concs,Keq_constant);
    Keq_inverse = np.power(Keq_constant,-1)
    KQ_r = me.odds(log_metabolites, mu0,-S_mat, P_mat, R_mat, delta_increment_for_small_concs,Keq_inverse,-1);
    
    epr = me.entropy_production_rate(KQ_f, KQ_r, E_regulation)

    delta_S_metab = me.calc_deltaS_metab(v_log_counts, target_v_log_counts);
    
    delta_S = me.calc_deltaS(v_log_counts,target_v_log_counts, f_log_counts, S_mat, KQ_f);
    
    [RR,Jac] = me.calc_Jac2(v_log_counts, f_log_counts, S_mat, delta_increment_for_small_concs, KQ_f, KQ_r, E_regulation)
    A = me.calc_A(v_log_counts, f_log_counts, S_mat, Jac, E_regulation )
        
    [ccc,fcc] = me.conc_flux_control_coeff(nvar, A, S_mat, rxn_flux, RR)

    if (React_Choice == -1) or (np.max(delta_S_metab)<=0):
        print("FINISHED OPTIMIZING")
        break
    if (np.sum(rxn_flux)<1.0):
        print("Unable to Optimize")
        break
    
    React_Choice = me.get_enzyme2regulate(ipolicy, delta_S_metab,delta_S,
                                        ccc, KQ_f, E_regulation,v_log_counts)
    
        
    newE = me.calc_reg_E_step(E_regulation, React_Choice, nvar, v_log_counts, 
                           f_log_counts, target_v_log_counts, S_mat, A, rxn_flux, KQ_f,
                           delta_S_metab)
        
    E_regulation[React_Choice] = newE
    #print("React_Choice")
    #print(reactions.index[React_Choice])
    #print(newE)
    i += 1
print("Activity Prediction")
display(E_regulation)

print("Enzymes Regulated")
display(active_reactions.index[E_regulation<1])

print("Flux")
display(rxn_flux)

FINISHED OPTIMIZING
Activity Prediction


array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 9.90352031e-04])

Enzymes Regulated


Index(['Pyruvate_Carboxylase'], dtype='object', name='REACTION')

Flux


array([0.63264797, 0.63264797, 0.63264797, 0.63264797, 0.63264797,
       1.26529595, 1.26529595, 1.26529595, 1.26529595, 1.26529595,
       1.26529595])

# ODE Simulations

## Calculate Rate Constants
Equation 9 from Cannon, et al, Prediction of Metabolite Concentrations, Rate Constants and Post-translational Regulation using Maximum Entropy-based Simulations with Application to Central Metabolism of Neurospora crassa," Processes 6, 63, DOI:10.3390/pr6060063.


In [17]:
def calculate_rate_constants(log_counts, rxn_flux,KQ_inverse, R, E_Regulation):
    KQ = np.power(KQ_inverse,-1)
    #Infer rate constants from reaction flux
    denominator = E_Regulation* np.exp(-R.dot(log_counts))*(1-KQ_inverse)
    # A reaction near equilibrium is problematic because (1-KQ_inverse)->0
    # By setting these reactions to be 
    # rate constant = 1/product_concs we are setting the rate to 1, which
    # is the same as the thermodynammic rate = KQ.
    one_idx, = np.where(KQ_inverse > 0.9)
    denominator[one_idx] = E_Regulation[one_idx]* np.exp(-R[one_idx,:].dot(log_counts));
    rxn_flux[one_idx] = 1;
    fwd_rate_constants = rxn_flux/denominator;
    
    return(fwd_rate_constants)

In [18]:
log_counts = np.append(v_log_counts,f_log_counts)
KQ_inverse = me.odds(log_counts,mu0,S_mat, R_mat, P_mat, delta_increment_for_small_concs, Keq_constant, direction = -1)
forward_rate_constants = calculate_rate_constants(log_counts, rxn_flux, KQ_inverse, R_mat, E_regulation)
reverse_rate_constants = forward_rate_constants/Keq_constant
display(forward_rate_constants)

array([3.80878308e+00, 7.71741611e+00, 2.32205942e-01, 9.74733866e+02,
       8.09991310e+01, 2.42680579e-04, 2.07210349e-08, 2.81356678e-01,
       3.85648060e-02, 4.27606386e-13, 6.10274262e-15])

In [19]:
display(reactions.index)

Index(['GLUCOSE_6_PHOSPHATASE', 'PGI', 'FBP', 'FBA', 'TPI', 'GAPD', 'PGK',
       'PGM', 'ENO', 'PEP_Carboxykinase', 'Pyruvate_Carboxylase', 'PYRt2m'],
      dtype='object', name='REACTION')

## Reinforcement Learning Based Regulation


In [20]:
#Setup neuran networ value function
#set variables in ML program

import machine_learning_functions as ml
import torch
device = torch.device("cpu")
gamma = 0.9

ml.device=device
ml.v_log_counts_static = v_log_counts_stationary
ml.target_v_log_counts = target_v_log_counts
ml.complete_target_log_counts = complete_target_log_counts
ml.Keq_constant = Keq_constant
ml.f_log_counts = f_log_counts

ml.P_mat = P_mat
ml.R_back_mat = R_mat
ml.S_mat = S_mat
ml.delta_increment_for_small_concs = delta_increment_for_small_concs
ml.nvar = nvar
ml.mu0 = mu0

ml.gamma = gamma
ml.num_rxns = Keq_constant.size

N, D_in, H, D_out = 1, Keq_constant.size,  4*Keq_constant.size, 1

#Make simple single hidden layer nn
nn_model = torch.nn.Sequential(
        torch.nn.Linear(D_in, H),
        torch.nn.Tanh(),
        torch.nn.Linear(H,D_out))

loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate=1e-4
optimizer = torch.optim.SGD(nn_model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=200, verbose=True, min_lr=1e-10,cooldown=10,threshold=5e-3)


In [21]:
#example of training 
maximum_episodes = 1 #test a single episode of training
epsilon_greedy=0.25 #percent of random steps
n=10 #n in n-step sarsa

episodic_reward = []
episodic_loss = []
episodic_loss_max = []
episodic_epr = []
final_states=np.zeros(Keq_constant.size)
final_KQ_fs=np.zeros(Keq_constant.size)
final_KQ_rs=np.zeros(Keq_constant.size)

for episode in range(0,maximum_episodes):
    
    state_sample = np.ones(Keq_constant.size)
        
    [sum_reward, average_loss, max_loss,final_epr,final_state,final_KQ_f,final_KQ_r,\
     reached_terminal_state, random_steps_taken,nn_steps_taken] = ml.sarsa_n(nn_model,loss_fn, optimizer, scheduler, state_sample, n, epsilon_greedy)
    
    if (reached_terminal_state):    
        episodic_reward.append(sum_reward)
        episodic_loss.append(average_loss)
        episodic_loss_max.append(max_loss)
        episodic_epr.append(final_epr)
        
        final_states = np.vstack((final_states,final_state))
        final_KQ_fs = np.vstack((final_KQ_fs,final_KQ_f))
        final_KQ_rs = np.vstack((final_KQ_rs,final_KQ_r))
        
    

**************************************Path Length ds<0******************************************
52
Final State
[6.400000e-01 4.096000e-01 8.000000e-01 5.120000e-01 6.400000e-01
 4.096000e-01 8.000000e-01 1.000000e+00 1.000000e+00 6.400000e-01
 6.338253e-04]
Final Flux
[0.41905144 0.41905144 0.41905144 0.41905144 0.41905144 0.83810288
 0.83810288 0.83810288 0.83810288 0.83810288 0.83810288]
final epr
0.5696445340710725
index of max error on path
42
